In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# Symbolic for derivatives

In [2]:
# 1/2 case.
vi, bi, o = sp.symbols('v_i b_i o')

Q = bi**(1/2)*vi/(bi**(1/2) + o) - bi
Q

b_i**0.5*v_i/(b_i**0.5 + o) - b_i

In [4]:
sp.diff(Q, bi)

0.5*v_i/(b_i**0.5*(b_i**0.5 + o)) - 0.5*v_i/(b_i**0.5 + o)**2 - 1

In [6]:
xi = sp.symbols('x_i')
expr = 1/2 *(-1 + 1/xi)
sp.diff(expr, xi)

-0.5/x_i**2

In [7]:
# alpha > 1
vi, xi = sp.symbols('v_i x_i')
a      = sp.symbols('a')
y      = sp.symbols('y')


t1 = vi*xi*a * (xi**(1-1/a) - xi**(2-1/a))
t2 = sp.integrate(vi*y*((a-1)*y**(-1/a)-(2*a-1)*y**(1-1/a)), (y, 0, xi))
uhat = t1 - t2

In [18]:
sp.simplify( uhat.evalf(subs={a:1.0001}))

v_i*(1.000000009998*x_i**1.000099990001 - 0.500025001249813*x_i**2.000099990001)

In [19]:
sp.simplify( uhat.evalf(subs={a:2}))

v_i*(1.33333333333333*x_i**1.5 - 0.8*x_i**2.5)

In [17]:
sp.simplify( uhat.evalf(subs={a:2/3}))

v_i*(1.33333333333333*x_i**0.5 - 0.444444444444445*x_i**1.5)

In [44]:
# derive normal way
v1, v2, b1, b2 = sp.symbols('v_1 v_2 b_1 b_2')

u1 = v1*b1**2 / (b1**2 + b2**2) - b1
u2 = v2*b2**2 / (b1**2 + b2**2) - b2
u1

b_1**2*v_1/(b_1**2 + b_2**2) - b_1

In [45]:
part1 = sp.simplify(sp.diff(u1, b1))
part2 = sp.simplify(sp.diff(u2, b2))
part1

2*b_1*b_2**2*v_1/(b_1**2 + b_2**2)**2 - 1

In [48]:
foc1 = sp.Eq(part1, 0)
foc2 = sp.Eq(part2, 0)
foc1

Eq(2*b_1*b_2**2*v_1/(b_1**2 + b_2**2)**2 - 1, 0)

In [65]:
sol = sp.solve((foc1.evalf(subs={v1:4}), foc2.evalf(subs={v2:2})), (b1, b2), dict=True)
sol

[{b_1: 1.28000000000000, b_2: 0.640000000000000}]

In [80]:
sp.simplify(sp.diff(part2, b2)).evalf(subs={v2:2, b1:32/25, b2: 16/25})

0.312500000000000

In [84]:
u2.evalf(subs={v2:2, b1:32/25})

2.0*b_2**2/(b_2**2 + 1.6384) - b_2

In [86]:
u1.evalf(subs={v1:4, b2:16/25})

4.0*b_1**2/(b_1**2 + 0.4096) - b_1

In [68]:
.evalf(subs={v2:2, b1:1.28, b2:0.64})sp.simplify(sp.diff(part2, b2))
sp.simplify(sp.diff(part1, b1)).evalf(subs={v1:4, b1:1.28, b2:0.64})

-1.71875000000000

In [6]:
# 1/2 case.
vi, xi = sp.symbols('v_i x_i')
y      = sp.symbols('y')


t1 = vi*xi*a * (xi**(1-1/a) - xi**(2-1/a))
t2 = sp.integrate(vi*y*((a-1)*y**(-1/a)-(2*a-1)*y**(1-1/a)), (y, 0, xi))
uhat = t1 - t2

In [9]:
uhat.evalf(subs={a:1/2+0.1})
uhat

a*v_i*x_i*(x_i**(1 - 1/a) - x_i**(2 - 1/a)) - Piecewise((v_i*(log(x_i)/3 + 2/(3*x_i)) - oo*sign(v_i), Eq(a, 1/3)), (-v_i*log(x_i)/2 - oo*sign(v_i), Eq(a, 1/2)), (v_i*(-4*a**3*x_i**2*x_i**(1/a)*x_i**(1 - 1/a)/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) + 3*a**3*x_i**2/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) + 4*a**2*x_i**2*x_i**(1/a)*x_i**(1 - 1/a)/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) - 4*a**2*x_i**2/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) - a*x_i**2*x_i**(1/a)*x_i**(1 - 1/a)/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) + a*x_i**2/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a))), True))